In [1]:
import praw
import re
import string
import pandas as pd
from collections import Counter
from multiprocessing import Process, Pipe
from praw.models import MoreComments
import nltk
from nltk.corpus import stopwords

In [2]:
nltk.download(['stopwords', 'words'])

[nltk_data] Downloading package stopwords to /Users/oscar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /Users/oscar/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

### Reddit API access
Once you create a Reddit app in the dev portal
retrieve the following params from it

In [3]:
APP_ID = 'ACqyZR3hYsXzfA'
APP_SECRET = 'xehsNEoOe_EH9DT-JwxekUDXLKxOIA'
USER_AGENT = 'my little scrapper'

### PRAW: Python Reddit API Wrapper
Does all the hard work
https://praw.readthedocs.io/en/latest/index.html

In [4]:
reddit = praw.Reddit(
     client_id=APP_ID,
     client_secret=APP_SECRET,
     user_agent=USER_AGENT,
     username="obiflow",
     password="Jmv94johgier!*8UwBC6kUBe"
)

Version 7.1.0 of praw is outdated. Version 7.2.0 was released Wednesday February 24, 2021.


In [5]:
subreddit = reddit.subreddit("wallstreetbets")
print(reddit.read_only)

False


### Short loop

only traverse the comment tree root
only retrieves top level comments

In [6]:
hot_submissions = {}

for submission in subreddit.hot(limit=5):
    print('Parsing comments for {}'.format(submission.title))
    author_name = submission.author.name
    author_redditor_instance = reddit.redditor(author_name)
    hot_submissions[submission.title] = {
    'comments': [comment for comment in submission.comments if not isinstance(comment, MoreComments)],
    'meta': {
        'score': submission.score,  # Output: the submission's score
        'submission_id': submission.id,     # Output: the submission's ID
        'submission_url': submission.url,   # Output: the URL the submission points to
        'author': {
            'name': author_name,
            'comment_karma': author_redditor_instance.comment_karma
        }
    }
    }

Parsing comments for What Are Your Moves Tomorrow, June 03, 2021
Parsing comments for Most Anticipated Earnings Releases for the week beginning May 31st, 2021
Parsing comments for I put 750 dollars in I have never had this much I know my dads looking down smiling thankyou apes
Parsing comments for To those who called me a r*tard for going all in on AMC 🤡🖕🏻
Parsing comments for We can’t stop... They can’t stop us, although they try... we’re true soldiers, we don’t die... We keep rolling... na-nah-nah-nah-nah!!!


### Long loop

Go through all of them

In [ ]:


hot_submissions = {}

for submission in subreddit.hot(limit=25):
    print('Parsing comments for {}'.format(submission.title))
    author_name = submission.author.name
    #author_redditor_instance = reddit.redditor(author_name)
    submission.comments.replace_more(limit=None)
    hot_submissions[submission.title] = {
    'comments': submission.comments.list(),
    'meta': {
        'score': submission.score,  # Output: the submission's score
        'submission_id': submission.id,     # Output: the submission's ID
        'submission_url': submission.url,   # Output: the URL the submission points to
        'author': {
            'name': author_name,
            'comment_karma': author_redditor_instance.comment_karma
        }
    }
    }

In [9]:
def get_author_comment_karma(author_name):
    redditor = reddit.redditor(author_name)
    comment_karma = redditor.comment_karma if redditor.comment_karma else None
    return comment_karma

def parse_comment(comment, title, score):
    if hasattr(comment, 'body'):
        author_name = comment.author.name if comment.author else None
        num_of_replies = 0
        if (len(comment.replies._comments) > 0):
            num_of_replies = comment.replies._comments[0].count if hasattr(comment.replies._comments[0], 'count') else 0
        parsed_comment = {
            'comment_id': comment.id,
            'thread': title,
            'body': comment.body,
            'author': author_name,
            'ups': comment.ups,
            'downs': comment.downs,
            'awards_count': comment.total_awards_received,
            'comment_score': comment.score,
            'created_at': comment.created,
            'num_of_replies': num_of_replies,
            'submission_score': comment.score
        }
        return parsed_comment
    else:
        return None

In [10]:
all_parsed_comments = []

for submission_title, submission_content in hot_submissions.items():
    comments = submission_content['comments']
    for comment in comments:
        parsed_comment = parse_comment(comment, submission_title, submission_content['meta']['score'])
        if parse_comment:
            all_parsed_comments.append(parsed_comment)

In [11]:
# first version, assumes you do not know the stocks by name
# uses regex to filter comment for exaclty three capital letters in a row
# with a space or a dollar sign before and a space after
# returns too many common words, not usable right now
# TODO look into exclusion with a word library

stock_name_re = '( \$[A-Z]{3,5} | [A-Z]{3,5} )'


bullish_words = ['hold', 'buy', 'buying', 'holding', 'yolo', 'long', 'bull', 'up', 'keep', 'moon', 'go', 'launch']
bearish_words = ['sell', 'short', 'shorting', 'selling', 'bear', 'down', 'crash', 'dump']

stock_mentions = {}
stock_mentions_doll = {}

stop_word_list = stopwords.words('english')


def sentiment_extraction(comment):
    text = comment.lower().translate(str.maketrans('', '', string.punctuation)).split()
    bullish_counter = Counter(w for w in text if w in bullish_words)
    bearish_counter = Counter(w for w in text if w in bearish_words)
    if sum(bearish_counter.values()) > sum(bullish_counter.values()):
        return 'bearish'
    else:
        return 'bullish'


for com in all_parsed_comments:
    body = ' '.join([word for word in com['body'].split() if word.lower() not in stop_word_list])
    com_stocks_names = re.findall(stock_name_re, body)
#     com_stocks_names_dol = re.findall(stock_name_re_doll, body)

    if len(com_stocks_names) > 0:
        for com_stock_name in com_stocks_names:
            name = com_stock_name.strip()
            if name[0] == '$':
                name = name[1:]
            if name in stock_mentions:
                stock_mentions[name]['count'] += 1
                stock_mentions[name]['score'].append(com['comment_score'])
                stock_mentions[name]['sentiment'].append(sentiment_extraction(body))
            else:
                stock_mentions[name] = {
                    'count': 1,
                    'score': [com['comment_score']],
                    'sentiment': [sentiment_extraction(body)]
                }


for title, thread_data in hot_submissions.items():
    com_stocks_names = re.findall(stock_name_re, title)
    print(title)
    if len(com_stocks_names) > 0:
        for com_stock_name in com_stocks_names:
            name = com_stock_name.strip()
            if name[0] == '$':
                name = name[1:]
            if name in stock_mentions:
                stock_mentions[name]['count'] += 1
                stock_mentions[name]['score'].append(thread_data['meta']['score'])
                stock_mentions[name]['sentiment'].append(sentiment_extraction(title))
            else:
                stock_mentions[name] = {}
                stock_mentions[name]['count'] = 1
                stock_mentions[name]['score'] = [thread_data['meta']['score']]
                stock_mentions[name]['sentiment'] = [sentiment_extraction(title)]


What Are Your Moves Tomorrow, June 03, 2021
Most Anticipated Earnings Releases for the week beginning May 31st, 2021
I put 750 dollars in I have never had this much I know my dads looking down smiling thankyou apes
To those who called me a r*tard for going all in on AMC 🤡🖕🏻
We can’t stop... They can’t stop us, although they try... we’re true soldiers, we don’t die... We keep rolling... na-nah-nah-nah-nah!!!


## Fine tuning

Guessing is not good enough, we import the word corpus to filter out all common words out of our dictionary. We also make sure the count is superior to five as a arbitrary sanity check to make sure this was recorded across multiple comments.

In [16]:
from nltk.corpus import words
word_list = words.words()

a = {key:data for key, data in stock_mentions.items() if key.lower() not in word_list and data['count'] > 5}

print("Stock name | Count | Score | Sentiment")
for key, data in a.items():
    print(key," | ", data['count']," | ", sum(data['score'])," | ", Counter(data['sentiment']).most_common(1)[0][0] )

Stock name | Count | Score | Sentiment
AMC  |  98  |  23265  |  bullish
TLRY  |  7  |  164  |  bullish
CLOV  |  8  |  170  |  bullish
GME  |  17  |  307  |  bullish


In [ ]:
for submission in subreddit.hot(limit=1):
    submission.comments.replace_more(limit=None)
    comment_queue = submission.comments[:]  # Seed with top-level
    while comment_queue:
        comment = comment_queue.pop(0)
        print(comment.body)
        comment_queue.extend(comment.replies)